In [50]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import os


In [51]:
load_dotenv()

True

In [52]:
model = ChatGoogleGenerativeAI(
    model = os.getenv("GEMINI_MODEL"),
    api_key = os.getenv("Google_API_KEY")
)

In [53]:
# create a state
class EvaluateBlogContent(TypedDict):
    title : str
    outline : str
    blog : str
    evaluate: float


In [54]:
def create_outline(state: EvaluateBlogContent) -> EvaluateBlogContent:
    title = state['title']
    prompt = f"create an outline for this title -> {title}"
    outline = model.invoke(prompt).content
    state['outline'] = outline
    return state
    

In [55]:
def create_blog(state: EvaluateBlogContent) -> EvaluateBlogContent:
    title = state['title']
    outline = state['outline']

    prompt = f"create a blog using this title {title} with this outline \n -> {outline}"
    
    blog = model.invoke(prompt).content
    state['blog'] = blog

    return state


In [56]:
def evaluation_score(state: EvaluateBlogContent) -> EvaluateBlogContent:
    title = state['title']
    outline = state['outline']
    blog = state['blog']

    prompt = f" Evaluate the blog {blog} outline {outline} and {title} provide me the match percentage between them i want output in percentage no any extra strngs to be there"
    percentage = model.invoke(prompt)

    state['evaluate'] = percentage
    return state



In [57]:
# create a graph
graph = StateGraph(EvaluateBlogContent)

# add nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)
graph.add_node('evaluation_score', evaluation_score)


# add edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', 'evaluation_score')
graph.add_edge('evaluation_score', END)


# compile
Workflow = graph.compile()

# execute
intial_state = {'title' : 'LPU'}
final_state = Workflow.invoke(intial_state)
print(final_state)



{'title': 'LPU', 'outline': 'Okay, here\'s an outline for the topic "LPU".  Since "LPU" most likely refers to Lovely Professional University, the outline will focus on that. If you were thinking of something else, please let me know and I can tailor the outline accordingly.\n\n**Outline: LPU (Lovely Professional University)**\n\n**I. Introduction**\n    *   A. Briefly define LPU:  Lovely Professional University, a private university in Punjab, India.\n    *   B. General overview of its significance and reputation.\n    *   C. Thesis statement:  LPU has emerged as a prominent university in India known for its diverse programs, infrastructure, and emphasis on practical learning, though it also faces certain challenges regarding perception and accessibility.\n\n**II. Academics and Programs**\n    *   A. Range of Disciplines:\n        *   1.  Engineering (mention specific branches)\n        *   2.  Management (MBA, BBA, etc.)\n        *   3.  Sciences (including agriculture)\n        *   4

In [58]:
print(final_state['outline'])

Okay, here's an outline for the topic "LPU".  Since "LPU" most likely refers to Lovely Professional University, the outline will focus on that. If you were thinking of something else, please let me know and I can tailor the outline accordingly.

**Outline: LPU (Lovely Professional University)**

**I. Introduction**
    *   A. Briefly define LPU:  Lovely Professional University, a private university in Punjab, India.
    *   B. General overview of its significance and reputation.
    *   C. Thesis statement:  LPU has emerged as a prominent university in India known for its diverse programs, infrastructure, and emphasis on practical learning, though it also faces certain challenges regarding perception and accessibility.

**II. Academics and Programs**
    *   A. Range of Disciplines:
        *   1.  Engineering (mention specific branches)
        *   2.  Management (MBA, BBA, etc.)
        *   3.  Sciences (including agriculture)
        *   4.  Arts, Humanities, and Social Sciences
   

In [59]:
print(final_state['blog'])

Okay, here's a blog post based on the outline provided, aiming for an informative and engaging read.  I've tried to balance promotional aspects with a realistic view of the university.

**LPU: A Deep Dive into Lovely Professional University**

Lovely Professional University (LPU), nestled in the heart of Punjab, India, has rapidly ascended to become a prominent name in the Indian higher education landscape.  More than just a university, LPU is a sprawling ecosystem of learning, innovation, and ambition.  But what truly defines LPU? This blog post delves into its academics, infrastructure, student life, and future prospects, offering a comprehensive overview of this dynamic institution. LPU has emerged as a prominent university in India known for its diverse programs, infrastructure, and emphasis on practical learning, though it also faces certain challenges regarding perception and accessibility.

**Academics: A Kaleidoscope of Disciplines**

LPU's academic offerings are impressively b

In [60]:
print(final_state['evaluate'])

content='95%' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--3c54554a-18c9-4032-a482-ec6bfa81b82f-0' usage_metadata={'input_tokens': 3022, 'output_tokens': 4, 'total_tokens': 3026, 'input_token_details': {'cache_read': 0}}
